# FunctionGemma Fine-tune với Unsloth

Train AI quyết định: `plant(plant_type, row, col)` hoặc `wait()`

**Ưu điểm Unsloth:**
- Nhanh hơn 2-5x so với HuggingFace
- Ít VRAM hơn (chạy được trên T4 free)
- Hỗ trợ FunctionGemma 270M native

**Output:** OpenVINO IR format cho inference nhanh

## 1. Cài đặt Unsloth

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install openvino optimum[openvino] -q

## 2. Upload Training Data

In [ ]:
from google.colab import files
import json

print("Upload training_data.json...")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    raw_data = json.load(f)

print(f"\n✓ Loaded {len(raw_data)} samples")
stats = {}
for s in raw_data:
    stats[s['action']] = stats.get(s['action'], 0) + 1
print(f"  Actions: {stats}")

## 3. Load FunctionGemma với Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/functiongemma-270m-it",
    max_seq_length=max_seq_length,
    load_in_4bit=False,   # Full precision cho model nhỏ
    load_in_16bit=True,   # 16bit LoRA
    full_finetuning=True, # Full finetune vì model nhỏ
)

print(f"✓ Model loaded: {model.config._name_or_path}")

## 4. Define Tools

In [ ]:
def plant(plant_type: str, row: int, col: int):
    """
    Plant a plant at grid position.

    Args:
        plant_type: Type of plant (pea_shooter, sunflower, wall_nut, cherry_bomb, etc)
        row: Row index 0-4 (0=top, 4=bottom)
        col: Column index 0-8 (0=left, 8=right)

    Returns:
        result: Action result
    """
    return {"result": "planted"}

def wait():
    """
    Wait and do nothing. Use when seed is on cooldown or no good action available.

    Returns:
        result: Action result
    """
    return {"result": "waiting"}

TOOLS = [plant, wait]
print("✓ Tools defined:", [f.__name__ for f in TOOLS])

## 5. Prepare Dataset

In [ ]:
from datasets import Dataset
import random

SYSTEM_MSG = """PvZ bot. Choose action based on game state.
- PLANTS: planted plants (type,row,col)
- ZOMBIES: zombies (type,row,col)
- SEEDS: seed packets (type,status: ready/cooldown)
Plant when seed ready and position valid. Wait when cooldown or no threat."""

def format_for_training(sample):
    """Format sample cho FunctionGemma chat template"""
    action = sample["action"]
    args = sample.get("arguments", {})

    if action == "plant":
        tool_call = {"type": "function", "function": {"name": "plant", "arguments": args}}
    else:
        tool_call = {"type": "function", "function": {"name": "wait", "arguments": {}}}

    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": sample["game_state"]},
        {"role": "assistant", "tool_calls": [tool_call]},
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tools=TOOLS,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": text}

# Shuffle và format
random.shuffle(raw_data)
dataset = Dataset.from_list(raw_data)
dataset = dataset.map(format_for_training, remove_columns=dataset.features)

# Split train/test
dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
print(f"✓ Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")
print(f"\nSample:\n{dataset['train'][0]['text'][:500]}...")

## 6. Training với Unsloth

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

num_samples = len(raw_data)
# Tự động điều chỉnh epochs dựa trên số samples
epochs = max(3, min(20, 500 // num_samples))

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        output_dir="pvz_gemma",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=epochs,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)

print(f"Training {num_samples} samples for {epochs} epochs...")
print(f"Batch size: 4 x 2 = 8 effective")

trainer_stats = trainer.train()
print(f"\n✓ Training complete in {trainer_stats.metrics['train_runtime']:.1f}s")

## 7. Test Model

In [ ]:
import re

def extract_tool_call(text):
    """Parse FunctionGemma output"""
    match = re.search(r"<start_function_call>call:(\w+)\{(.*?)\}<end_function_call>", text, re.DOTALL)
    if not match:
        return None
    name = match.group(1)
    args_str = match.group(2)
    args = {}
    for k, v in re.findall(r"(\w+):([^,}]+)", args_str):
        v = v.strip()
        try:
            args[k] = int(v)
        except:
            args[k] = v
    return {"name": name, "arguments": args}

def test_bot(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tools=TOOLS, add_generation_prompt=True,
        return_dict=True, return_tensors="pt"
    )
    out = model.generate(
        **inputs.to(model.device),
        max_new_tokens=64,
        top_k=64, top_p=0.95, temperature=1.0,
        pad_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)
    return extract_tool_call(output)

print("="*50)
print("TEST PVZ BOT")
print("="*50)

test_cases = [
    "PLANTS:[]. ZOMBIES:[]. SEEDS:[(pea_shooter,ready)]",
    "PLANTS:[(pea_shooter,2,0)]. ZOMBIES:[(zombie,2,7)]. SEEDS:[(pea_shooter,cooldown)]",
    "PLANTS:[(pea_shooter,2,0)]. ZOMBIES:[(zombie,1,6)]. SEEDS:[(pea_shooter,ready)]",
    "PLANTS:[]. ZOMBIES:[(zombie,0,8),(zombie,4,7)]. SEEDS:[(pea_shooter,ready),(sunflower,ready)]",
]

for t in test_cases:
    result = test_bot(t)
    print(f"\n📥 {t}")
    print(f"📤 {result}")

## 8. Save Model

In [ ]:
# Save với Unsloth (nhanh hơn)
model.save_pretrained("pvz_gemma_pytorch")
tokenizer.save_pretrained("pvz_gemma_pytorch")
print("✓ PyTorch model saved to pvz_gemma_pytorch/")

## 9. Export to OpenVINO IR

In [ ]:
from optimum.intel import OVModelForCausalLM

print("Converting to OpenVINO IR format...")
ov_model = OVModelForCausalLM.from_pretrained(
    "pvz_gemma_pytorch",
    export=True,
    compile=False
)
ov_model.save_pretrained("pvz_gemma_openvino")
tokenizer.save_pretrained("pvz_gemma_openvino")
print("✓ OpenVINO model saved to pvz_gemma_openvino/")

## 10. Test OpenVINO Model

In [ ]:
from optimum.intel import OVModelForCausalLM
from transformers import AutoTokenizer

ov_model = OVModelForCausalLM.from_pretrained("pvz_gemma_openvino")
ov_tokenizer = AutoTokenizer.from_pretrained("pvz_gemma_openvino")

def test_ov(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    inputs = ov_tokenizer.apply_chat_template(
        messages, tools=TOOLS, add_generation_prompt=True,
        return_dict=True, return_tensors="pt"
    )
    out = ov_model.generate(
        **inputs,
        max_new_tokens=64,
        pad_token_id=ov_tokenizer.eos_token_id
    )
    output = ov_tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)
    return extract_tool_call(output)

print("\n" + "="*50)
print("TEST OPENVINO MODEL")
print("="*50)

for t in test_cases:
    result = test_ov(t)
    print(f"\n📥 {t}")
    print(f"📤 {result}")

## 11. Download Models

In [ ]:
!zip -r pvz_gemma_openvino.zip pvz_gemma_openvino/

print("\n✓ Model ready for download:")
print("  - pvz_gemma_openvino.zip (OpenVINO IR format)")

# Show size
!ls -lh pvz_gemma_openvino.zip

In [ ]:
from google.colab import files
files.download('pvz_gemma_openvino.zip')

## (Optional) Save to HuggingFace Hub

In [ ]:
# Uncomment để push lên HuggingFace
# from huggingface_hub import login
# login()
# model.push_to_hub("your-username/pvz-gemma-bot")
# tokenizer.push_to_hub("your-username/pvz-gemma-bot")